# Multiple Adjacency Spectral Embedding (MASE)

Multiple Adjacency Spectral Embedding (MASE) is an extension of Adjacency Spectral Embedding (see ASE [tutorial](https://graspy.neurodata.io/tutorials/embedding/adjacencyspectralembed)) for an arbitrary number of graphs.  Once graphs are embedded, the low-dimensional Euclidean representation can be used to visualize the latent positions of vertices, perform inference, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Replicating Omnibus Embedding Tutorial

In order to demonstrate how to use the ``MultipleASE`` class, we will use a simple example: two 2-block stochastic block models (SBMs) with different block probabilities.  Indeed, we will use the same models as in the omnibus [tutorial](https://graspy.neurodata.io/tutorials/embedding/omnibus) to facilitate direct comparison. 

## Simulate two different graphs using stochastic block models (SBM)

We sample two 2-block SBMs (undirected, no self-loops) with 50 vertices, each block containing 25 vertices ($n = [25, 25]$), and with the following block connectivity matrices:

\begin{align*}
B_1 = 
\begin{bmatrix}0.3 & 0.1\\
0.1 & 0.7
\end{bmatrix},~
B_2 = \begin{bmatrix}0.3 & 0.1\\
0.1 & 0.3
\end{bmatrix}
\end{align*}

The only difference between the two is the within-block probability for the second block. We sample $G_1 \sim \text{SBM}(n, B_1)$ and $G_2 \sim \text{SBM}(n, B_2)$.

In [ ]:
from graspologic.simulations import sbm

n = [25, 25]
B1 = [[0.3, 0.1],
      [0.1, 0.7]]
B2 = [[0.3, 0.1],
      [0.1, 0.3]]

np.random.seed(8)
G1 = sbm(n, B1)
G2 = sbm(n, B2)

## Visualize the graphs using heatmap

We visualize the sampled graphs using the ``heatmap()`` function. ``heatmap()`` will plot the adjacency matrix, where the colors represent the weight of each edge. In this case, we have binary graphs so the values will be either 0 or 1. 

We see that there is clear block structure to the graphs. Furthermore, the lower right quarter of $G_1$, representing the within-group connections for the second group, is more dense than that of $G_2$.

In [ ]:
from graspologic.plot import heatmap

heatmap(G1, figsize=(7,7), title="Visualization of Graph 1", cbar=False)
_ = heatmap(G2, figsize=(7,7), title="Visualization of Graph 2", cbar=False)

## Embed the two graphs using MASE

Just as ASE fits a single input graph to the Random Dot Product Graph (RDPG) model, MASE fits many input graphs to the common subspace independent-edge (COSIE) model, which is an extension of the RDPG model to multiple graphs. The COSIE model assumes that all of the graphs have important shared properties, but can also have meaningful differences.  As a result, COSIE decomposes the set of graphs into a set of shared latent positions $V$ that describe similarities, and a set of score matrices $R^{(i)}$ that describe how each individual graph is different.  Mathematically, for each graph $A^{(i)}$,
$$A^{(i)} = VR^{(i)}V^T$$

In general, 2 clusters can be faithfully represented in 2 dimensions, so we specify the number of components as 2. Then, the ``fit_transform()`` method returns the estimated latent positions $\hat{V}$, which we can see has 50 entries corresponding to the 50 vertices embedded in 2 dimensions.

In [ ]:
from graspologic.embed import MultipleASE as MASE

embedder = MASE(n_components=2)
V_hat = embedder.fit_transform([G1, G2])
print(V_hat.shape)

## Visualize Common Latent Subspace

Since the two graphs have clear block structures, with higher within-block probabilities than between-block, we see two distinct "clusters" when we visualize the latent subspace positions.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(V_hat[:25, 0], V_hat[:25, 1], c="blue", label="Block 1")
ax.scatter(V_hat[25:, 0], V_hat[25:, 1], c="red", label="Block 2")
ax.legend(prop={'size':20})
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xticks([])
ax.set_yticks([])
_ = ax.set_title("Common Latent Subspace from MASE Embedding", fontsize=20)

## Recover Probability Matrices

Since ``MultipleASE()`` stores estimates of the individual score matrices $\hat{R}^{(i)}$ as well as an estimate of the common subspace positions $\hat{V}$, we can recover an estimate of the probability matrix for each graph 
$$\hat{P}^{(i)} = \hat{V} \hat{R}^{(i)} \hat{V}^T$$

In [ ]:
R = embedder.scores_
R_1 = R[0]
R_2 = R[1]

V_hat_T = np.transpose(V_hat)
P_1_hat = np.matmul(np.matmul(V_hat, R_1), V_hat_T)
P_2_hat = np.matmul(np.matmul(V_hat, R_2), V_hat_T)

P1_hat_vis = heatmap(P_1_hat, figsize=(7,7), title="Visualization of Probability Matrix 1", vmin=0, vmax=1)
P2_hat_vis = heatmap(P_2_hat, figsize=(7,7), title="Visualization of Probability Matrix 2", vmin=0, vmax=1)

## Recover Original Block Connectivity Matrices

Now, we can see the clear block structure once again.  In order to obtain an estmate of the original block connectivity matrices, we can average across the blocks.

In [ ]:
B1_hat = np.array([[np.mean(P_1_hat[:25, :25]), np.mean(P_1_hat[:25, 25:])],
               [np.mean(P_1_hat[25:, :25]), np.mean(P_1_hat[25:, 25:])]])
B2_hat = np.array([[np.mean(P_2_hat[:25, :25]), np.mean(P_2_hat[:25, 25:])],
               [np.mean(P_2_hat[25:, :25]), np.mean(P_2_hat[25:, 25:])]])
print("B1_hat =\n", B1_hat)
print("B2_hat =\n", B2_hat)

 Although we don't get exactly the same block probabilities as we input into the SBM, it's fairly close with only one 50-vertex graph for each!

## Sampling More Graphs

We can recover an even more accurate representation of the two SBMs with more sample graphs.  Let's try with 100 graphs sampled from each.

In [ ]:
G1 = [sbm(n, B1) for i in range(100)]
G2 = [sbm(n, B2) for i in range(100)]

V_hat = embedder.fit_transform(G1 + G2)

## Visualizing Common Latent Subspace

Now, the clustering is even more distinct, as we would expect with more samples.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(V_hat[:25, 0], V_hat[:25, 1], c="blue", label="Block 1")
ax.scatter(V_hat[25:, 0], V_hat[25:, 1], c="red", label="Block 2")
ax.legend(prop={'size':20})
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Common Latent Subspace from MASE Embedding with Many Sampled Graphs", fontsize=20);

## Recover Probability Matrices

Since we have many samples, we first obtain the estimated probability matrix $\hat{P}^{(i)} = \hat{V} \hat{R}^{(i)} \hat{V}^T$ for each graph, and then we average across all the graphs generated from the same SBM in order to recover the probabiliity matrices.

In [ ]:
#Obtain list of P_hats for each graph
R = embedder.scores_
V_hat_T = np.transpose(V_hat)
P_hat = np.array([np.matmul(np.matmul(V_hat, R[i]), V_hat_T) for i in range(len(R))])

#Average across all graphs generated from the same distribution.
P1_hat = np.mean(P_hat[:100], 0)
P2_hat = np.mean(P_hat[100:], 0)

#Visualize
P1_hat_vis = heatmap(P1_hat, figsize=(7,7), title="Visualization of Probability Matrix 1", vmin=0, vmax=1)
P2_hat_vis = heatmap(P2_hat, figsize=(7,7), title="Visualization of Probability Matrix 2", vmin=0, vmax=1)

## Recover Original Block Connectivity Matrices

And we see that our estimates for the block probabilities are closer to the true values with a higher number of samples.

In [ ]:
B1_hat = np.array([[np.mean(P1_hat[:25, :25]), np.mean(P1_hat[:25, 25:])],
               [np.mean(P_1_hat[25:, :25]), np.mean(P_1_hat[25:, 25:])]])
B2_hat = np.array([[np.mean(P2_hat[:25, :25]), np.mean(P2_hat[:25, 25:])],
               [np.mean(P2_hat[25:, :25]), np.mean(P2_hat[25:, 25:])]])
print("B1_hat =\n", B1_hat)
print("B1_hat =\n", B2_hat)